In [1]:
import pandas as pd

In [2]:
fips_url_full = 'https://www.dropbox.com/s/04eu0q6sgph8wto/US_FIPS_Codes.xls?dl=1'
fips_url_abv = 'https://www.dropbox.com/s/d503mesxlsc1yfa/county_fips.csv?dl=1'
fl_pop_path = '../20_intermediate_files/fl_shipment_pop.csv'
fl_path = '../20_intermediate_files/florida_shipment_cleaned.csv'

In [3]:
fl_df = pd.read_csv(fl_path)
fl_df.head()

,BUYER_STATE,BUYER_COUNTY,YEAR,MME
0,AZ,APACHE,2006,4.956969e+06
1,AZ,APACHE,2007,5.611684e+06
2,AZ,APACHE,2008,5.853244e+06
3,AZ,APACHE,2009,7.344332e+06
4,AZ,APACHE,2010,7.732743e+06


In [4]:
fips_full = pd.read_excel(fips_url_full, header=1, dtype={'State':str, 'County Name':str,'FIPS State':str, 'FIPS County':str})
fips_full['FIPS Code'] = fips_full['FIPS State'] + fips_full['FIPS County']
fips_full['FIPS Code'] = fips_full['FIPS Code'].str.lstrip('0')
fips_full.drop(columns=['FIPS State', 'FIPS County'], inplace=True)
fips_full['FIPS Code'] = fips_full['FIPS Code'].astype(int)
fips_full.head()

,State,County Name,FIPS Code
0,Alabama,Autauga,1001
1,Alabama,Baldwin,1003
2,Alabama,Barbour,1005
3,Alabama,Bibb,1007
4,Alabama,Blount,1009


In [5]:
fips_abv = pd.read_csv(fips_url_abv)
fips_abv.rename(columns={'countyfips': 'FIPS Code'}, inplace=True)
fips_abv

,BUYER_COUNTY,BUYER_STATE,FIPS Code
0,AUTAUGA,AL,1001
1,BALDWIN,AL,1003
2,BARBOUR,AL,1005
3,BIBB,AL,1007
4,BLOUNT,AL,1009
...,...,...,...
3138,WASHAKIE,WY,56043
3139,WESTON,WY,56045
3140,SKAGWAY,AK,2230
3141,HOONAH ANGOON,AK,2105


In [6]:
fips = fips_abv.merge(fips_full, how = 'left', on=['FIPS Code'], validate="1:1")


In [7]:

fips.drop(columns=['County Name'],inplace=True)
#NaN for AK
fips.isna().sum()

BUYER_COUNTY    0
BUYER_STATE     0
FIPS Code       0
State           3
dtype: int64

In [8]:
missing_state = fips['State'].isna()
missing_state

0       False
1       False
2       False
3       False
4       False
        ...  
3138    False
3139    False
3140     True
3141     True
3142     True
Name: State, Length: 3143, dtype: bool

In [9]:
impute_dict=dict({'AK':'Arkansas'})
fips.loc[missing_state, 'State'] = fips.loc[missing_state, 'BUYER_STATE'].map(impute_dict)
print(fips.isna().sum())
fips.shape

BUYER_COUNTY    0
BUYER_STATE     0
FIPS Code       0
State           0
dtype: int64


(3143, 4)

In [10]:
fl_ship_fips = fl_df.merge(fips, how='left', on=['BUYER_STATE','BUYER_COUNTY'], validate="m:1")
fl_ship_fips

,BUYER_STATE,BUYER_COUNTY,YEAR,MME,FIPS Code,State
0,AZ,APACHE,2006,4.956969e+06,4001,Arizona
1,AZ,APACHE,2007,5.611684e+06,4001,Arizona
2,AZ,APACHE,2008,5.853244e+06,4001,Arizona
3,AZ,APACHE,2009,7.344332e+06,4001,Arizona
4,AZ,APACHE,2010,7.732743e+06,4001,Arizona
...,...,...,...,...,...,...
1715,SC,YORK,2010,5.209517e+07,45091,South Carolina
1716,SC,YORK,2011,5.296504e+07,45091,South Carolina
1717,SC,YORK,2012,5.644417e+07,45091,South Carolina
1718,SC,YORK,2013,5.813704e+07,45091,South Carolina


In [11]:
fl_df_pop = pd.read_csv(fl_pop_path)
fl_df_pop['STNAME'].unique()


array(['Arizona', 'Florida', 'Louisiana', 'South Carolina'], dtype=object)

In [12]:
fl_df_pop['CTYNAME'] = fl_df_pop['CTYNAME'].str.replace(' County', '')
fl_df_pop['CTYNAME'] = fl_df_pop['CTYNAME'].str.upper()
fl_df_pop

,STNAME,CTYNAME,YEAR,POPULATION
0,Arizona,APACHE,2000,69516.0
1,Arizona,COCHISE,2000,118028.0
2,Arizona,COCONINO,2000,116717.0
3,Arizona,GILA,2000,51355.0
4,Arizona,GRAHAM,2000,33541.0
...,...,...,...,...
3855,South Carolina,SPARTANBURG,2019,320254.0
3856,South Carolina,SUMTER,2019,106635.0
3857,South Carolina,UNION,2019,27221.0
3858,South Carolina,WILLIAMSBURG,2019,30256.0


In [123]:
fl_df_pop.loc[fl_df_pop["STNAME"] == "Louisiana", "CTYNAME"].str.replace(' PARISH', '', inplace=True)
fl_df_pop.loc[fl_df_pop.STNAME == 'Louisiana', 'CTYNAME'].unique()

TypeError: StringMethods.replace() got an unexpected keyword argument 'inplace'

In [13]:
fl_ship_fips

,BUYER_STATE,BUYER_COUNTY,YEAR,MME,FIPS Code,State
0,AZ,APACHE,2006,4.956969e+06,4001,Arizona
1,AZ,APACHE,2007,5.611684e+06,4001,Arizona
2,AZ,APACHE,2008,5.853244e+06,4001,Arizona
3,AZ,APACHE,2009,7.344332e+06,4001,Arizona
4,AZ,APACHE,2010,7.732743e+06,4001,Arizona
...,...,...,...,...,...,...
1715,SC,YORK,2010,5.209517e+07,45091,South Carolina
1716,SC,YORK,2011,5.296504e+07,45091,South Carolina
1717,SC,YORK,2012,5.644417e+07,45091,South Carolina
1718,SC,YORK,2013,5.813704e+07,45091,South Carolina


In [14]:
fl_ship_pop = fl_ship_fips.merge(fl_df_pop, how='left', left_on=['State', 'BUYER_COUNTY','YEAR'], right_on=['STNAME', 'CTYNAME', 'YEAR'], validate="1:1")

In [15]:
fl_ship_pop

,BUYER_STATE,BUYER_COUNTY,YEAR,MME,FIPS Code,State,STNAME,CTYNAME,POPULATION
0,AZ,APACHE,2006,4.956969e+06,4001,Arizona,Arizona,APACHE,69066.0
1,AZ,APACHE,2007,5.611684e+06,4001,Arizona,Arizona,APACHE,69278.0
2,AZ,APACHE,2008,5.853244e+06,4001,Arizona,Arizona,APACHE,69520.0
3,AZ,APACHE,2009,7.344332e+06,4001,Arizona,Arizona,APACHE,70591.0
4,AZ,APACHE,2010,7.732743e+06,4001,Arizona,Arizona,APACHE,71828.0
...,...,...,...,...,...,...,...,...,...
1715,SC,YORK,2010,5.209517e+07,45091,South Carolina,South Carolina,YORK,226871.0
1716,SC,YORK,2011,5.296504e+07,45091,South Carolina,South Carolina,YORK,230131.0
1717,SC,YORK,2012,5.644417e+07,45091,South Carolina,South Carolina,YORK,234151.0
1718,SC,YORK,2013,5.813704e+07,45091,South Carolina,South Carolina,YORK,238736.0


In [16]:
wa_path = '../20_intermediate_files/washington_shipment_cleaned.csv'
wa_pop_path = '../20_intermediate_files/wa_shipment_pop.csv'

In [17]:
wa_df = pd.read_csv(wa_path)
wa_ship_fips = wa_df.merge(fips, how="left", on=["BUYER_STATE", "BUYER_COUNTY"], validate="m:1")
wa_ship_fips

,BUYER_STATE,BUYER_COUNTY,YEAR,MME,FIPS Code,State
0,AZ,APACHE,2006,4.956969e+06,4001,Arizona
1,AZ,APACHE,2007,5.611684e+06,4001,Arizona
2,AZ,APACHE,2008,5.853244e+06,4001,Arizona
3,AZ,APACHE,2009,7.344332e+06,4001,Arizona
4,AZ,APACHE,2010,7.732743e+06,4001,Arizona
...,...,...,...,...,...,...
1565,WA,YAKIMA,2010,6.746887e+07,53077,Washington
1566,WA,YAKIMA,2011,7.600735e+07,53077,Washington
1567,WA,YAKIMA,2012,8.023265e+07,53077,Washington
1568,WA,YAKIMA,2013,8.391631e+07,53077,Washington


In [18]:

#clean and standardize population data to merge with shipment
wa_df_pop = pd.read_csv(wa_pop_path)
wa_df_pop['CTYNAME'] = wa_df_pop['CTYNAME'].str.replace(' County', '')
wa_df_pop['CTYNAME'] = wa_df_pop['CTYNAME'].str.upper()
wa_df_pop.loc[wa_df_pop['CTYNAME'] == 'ST. LAWRENCE', 'CTYNAME'] = 'SAINT LAWRENCE'
wa_df_pop[wa_df_pop.CTYNAME == 'SAINT LAWRENCE']

,STNAME,CTYNAME,YEAR,POPULATION
1624,New York,SAINT LAWRENCE,2000,111869
1686,New York,SAINT LAWRENCE,2001,111202
1748,New York,SAINT LAWRENCE,2002,110686
1810,New York,SAINT LAWRENCE,2003,110381
1872,New York,SAINT LAWRENCE,2004,110224
1934,New York,SAINT LAWRENCE,2005,110072
1996,New York,SAINT LAWRENCE,2006,109769
2058,New York,SAINT LAWRENCE,2007,109594
2120,New York,SAINT LAWRENCE,2008,109553
2182,New York,SAINT LAWRENCE,2009,109715


In [19]:

#merge shipment and population data
wa_ship_pop = wa_ship_fips.merge(wa_df_pop, how='left', left_on=['State', 'BUYER_COUNTY','YEAR'], right_on=['STNAME', 'CTYNAME', 'YEAR'], copy=False)
wa_ship_pop.isna().sum()

BUYER_STATE     0
BUYER_COUNTY    0
YEAR            0
MME             0
FIPS Code       0
State           0
STNAME          0
CTYNAME         0
POPULATION      0
dtype: int64

In [20]:
wa_ship = pd.read_csv('../20_intermediate_files/wa_ship_merge.csv')
wa_ship.isna().sum()

BUYER_STATE     0
BUYER_COUNTY    0
YEAR            0
MME             0
FIPS Code       0
State           0
STNAME          9
CTYNAME         9
POPULATION      9
dtype: int64

In [21]:
fl_ship = pd.read_csv('../20_intermediate_files/fl_ship_merge.csv')
fl_ship.isna().sum()

BUYER_STATE       0
BUYER_COUNTY      0
YEAR              0
MME               0
FIPS Code         0
State             0
STNAME          598
CTYNAME         598
POPULATION      598
dtype: int64

In [22]:
wa_ship[wa_ship.STNAME.isna()]
#population does not have SAINT LAWRENCE, > ST. LAWRENCE

,BUYER_STATE,BUYER_COUNTY,YEAR,MME,FIPS Code,State,STNAME,CTYNAME,POPULATION
1057,NY,SAINT LAWRENCE,2006,2.186481e+07,36089,New York,NaN,NaN,NaN
1058,NY,SAINT LAWRENCE,2007,2.305936e+07,36089,New York,NaN,NaN,NaN
1059,NY,SAINT LAWRENCE,2008,2.414880e+07,36089,New York,NaN,NaN,NaN
1060,NY,SAINT LAWRENCE,2009,2.740978e+07,36089,New York,NaN,NaN,NaN
1061,NY,SAINT LAWRENCE,2010,2.894361e+07,36089,New York,NaN,NaN,NaN
1062,NY,SAINT LAWRENCE,2011,3.018991e+07,36089,New York,NaN,NaN,NaN
1063,NY,SAINT LAWRENCE,2012,2.992549e+07,36089,New York,NaN,NaN,NaN
1064,NY,SAINT LAWRENCE,2013,3.069736e+07,36089,New York,NaN,NaN,NaN
1065,NY,SAINT LAWRENCE,2014,3.062440e+07,36089,New York,NaN,NaN,NaN


In [23]:
fips

,BUYER_COUNTY,BUYER_STATE,FIPS Code,State
0,AUTAUGA,AL,1001,Alabama
1,BALDWIN,AL,1003,Alabama
2,BARBOUR,AL,1005,Alabama
3,BIBB,AL,1007,Alabama
4,BLOUNT,AL,1009,Alabama
...,...,...,...,...
3138,WASHAKIE,WY,56043,Wyoming
3139,WESTON,WY,56045,Wyoming
3140,SKAGWAY,AK,2230,Arkansas
3141,HOONAH ANGOON,AK,2105,Arkansas


In [25]:
wa_df_pop

,STNAME,CTYNAME,YEAR,POPULATION
0,Arizona,APACHE,2000,69516
1,Arizona,COCHISE,2000,118028
2,Arizona,COCONINO,2000,116717
3,Arizona,GILA,2000,51355
4,Arizona,GRAHAM,2000,33541
...,...,...,...,...
3595,Washington,WAHKIAKUM,2019,4420
3596,Washington,WALLA WALLA,2019,61062
3597,Washington,WHATCOM,2019,228675
3598,Washington,WHITMAN,2019,50136


In [33]:
wa_pop_fips = wa_df_pop.merge(fips, how='left', left_on=['STNAME', 'CTYNAME'], right_on = ['State', 'BUYER_COUNTY'])
wa_pop_fips 

,STNAME,CTYNAME,YEAR,POPULATION,BUYER_COUNTY,BUYER_STATE,FIPS Code,State
0,Arizona,APACHE,2000,69516,APACHE,AZ,4001,Arizona
1,Arizona,COCHISE,2000,118028,COCHISE,AZ,4003,Arizona
2,Arizona,COCONINO,2000,116717,COCONINO,AZ,4005,Arizona
3,Arizona,GILA,2000,51355,GILA,AZ,4007,Arizona
4,Arizona,GRAHAM,2000,33541,GRAHAM,AZ,4009,Arizona
...,...,...,...,...,...,...,...,...
3595,Washington,WAHKIAKUM,2019,4420,WAHKIAKUM,WA,53069,Washington
3596,Washington,WALLA WALLA,2019,61062,WALLA WALLA,WA,53071,Washington
3597,Washington,WHATCOM,2019,228675,WHATCOM,WA,53073,Washington
3598,Washington,WHITMAN,2019,50136,WHITMAN,WA,53075,Washington


In [34]:
wa_pop_fips = wa_pop_fips[['STNAME', 'CTYNAME', 'FIPS Code', 'YEAR', 'POPULATION']]
wa_pop_fips

,STNAME,CTYNAME,FIPS Code,YEAR,POPULATION
0,Arizona,APACHE,4001,2000,69516
1,Arizona,COCHISE,4003,2000,118028
2,Arizona,COCONINO,4005,2000,116717
3,Arizona,GILA,4007,2000,51355
4,Arizona,GRAHAM,4009,2000,33541
...,...,...,...,...,...
3595,Washington,WAHKIAKUM,53069,2019,4420
3596,Washington,WALLA WALLA,53071,2019,61062
3597,Washington,WHATCOM,53073,2019,228675
3598,Washington,WHITMAN,53075,2019,50136


In [37]:
wa_ship_fips = wa_ship_fips[['BUYER_STATE', 'BUYER_COUNTY', 'FIPS Code', 'YEAR', 'MME', ]]
wa_ship_fips

,BUYER_STATE,BUYER_COUNTY,FIPS Code,YEAR,MME
0,AZ,APACHE,4001,2006,4.956969e+06
1,AZ,APACHE,4001,2007,5.611684e+06
2,AZ,APACHE,4001,2008,5.853244e+06
3,AZ,APACHE,4001,2009,7.344332e+06
4,AZ,APACHE,4001,2010,7.732743e+06
...,...,...,...,...,...
1565,WA,YAKIMA,53077,2010,6.746887e+07
1566,WA,YAKIMA,53077,2011,7.600735e+07
1567,WA,YAKIMA,53077,2012,8.023265e+07
1568,WA,YAKIMA,53077,2013,8.391631e+07


In [42]:
wa_ship_pop = wa_ship_fips.merge(wa_pop_fips, how='left', on=['FIPS Code', 'YEAR'])

,BUYER_STATE,BUYER_COUNTY,FIPS Code,YEAR,MME,STNAME,CTYNAME,POPULATION
0,AZ,APACHE,4001,2006,4.956969e+06,Arizona,APACHE,69066
1,AZ,APACHE,4001,2007,5.611684e+06,Arizona,APACHE,69278
2,AZ,APACHE,4001,2008,5.853244e+06,Arizona,APACHE,69520
3,AZ,APACHE,4001,2009,7.344332e+06,Arizona,APACHE,70591
4,AZ,APACHE,4001,2010,7.732743e+06,Arizona,APACHE,71828
...,...,...,...,...,...,...,...,...
1565,WA,YAKIMA,53077,2010,6.746887e+07,Washington,YAKIMA,244249
1566,WA,YAKIMA,53077,2011,7.600735e+07,Washington,YAKIMA,245899
1567,WA,YAKIMA,53077,2012,8.023265e+07,Washington,YAKIMA,246064
1568,WA,YAKIMA,53077,2013,8.391631e+07,Washington,YAKIMA,246395


In [109]:
fl_df_pop.isna().sum()

STNAME         0
CTYNAME        0
YEAR           0
POPULATION    20
dtype: int64

In [ ]:
fl_df_pop

In [107]:
fl_pop_fips = fl_df_pop.merge(fips, how='left', left_on=['STNAME', 'CTYNAME'], right_on = ['State', 'BUYER_COUNTY'])
fl_pop_fips = fl_pop_fips[['STNAME', 'CTYNAME', 'FIPS Code', 'YEAR', 'POPULATION']]
fl_pop_fips.isna().sum()

STNAME           0
CTYNAME          0
FIPS Code     1360
YEAR             0
POPULATION      20
dtype: int64

In [102]:
fl_ship_fips = fl_ship_fips[['BUYER_STATE', 'BUYER_COUNTY', 'FIPS Code', 'YEAR', 'MME', ]]
fl_ship_fips#.BUYER_STATE.unique()
fl_ship_fips[fl_ship_fips['FIPS Code'] == ]

,BUYER_STATE,BUYER_COUNTY,FIPS Code,YEAR,MME
0,AZ,APACHE,4001,2006,4.956969e+06
1,AZ,APACHE,4001,2007,5.611684e+06
2,AZ,APACHE,4001,2008,5.853244e+06
3,AZ,APACHE,4001,2009,7.344332e+06
4,AZ,APACHE,4001,2010,7.732743e+06
5,AZ,APACHE,4001,2011,7.855511e+06
6,AZ,APACHE,4001,2012,8.448550e+06
7,AZ,APACHE,4001,2013,9.726018e+06
8,AZ,APACHE,4001,2014,1.075259e+07


In [118]:
fips[fips.BUYER_COUNTY.str.contains('WEST CARROLL')]

,BUYER_COUNTY,BUYER_STATE,FIPS Code,State
1170,WEST CARROLL,LA,22123,Louisiana


In [112]:
fl_pop_fips[fl_pop_fips.STNAME == 'Louisiana']['CTYNAME'].unique()


array(['ACADIA PARISH', 'ALLEN PARISH', 'ASCENSION PARISH',
       'ASSUMPTION PARISH', 'AVOYELLES PARISH', 'BEAUREGARD PARISH',
       'BIENVILLE PARISH', 'BOSSIER PARISH', 'CADDO PARISH',
       'CALCASIEU PARISH', 'CALDWELL PARISH', 'CAMERON PARISH',
       'CATAHOULA PARISH', 'CLAIBORNE PARISH', 'CONCORDIA PARISH',
       'DE SOTO PARISH', 'EAST BATON ROUGE PARISH', 'EAST CARROLL PARISH',
       'EAST FELICIANA PARISH', 'EVANGELINE PARISH', 'FRANKLIN PARISH',
       'GRANT PARISH', 'IBERIA PARISH', 'IBERVILLE PARISH',
       'JACKSON PARISH', 'JEFFERSON PARISH', 'JEFFERSON DAVIS PARISH',
       'LAFAYETTE PARISH', 'LAFOURCHE PARISH', 'LA SALLE PARISH',
       'LINCOLN PARISH', 'LIVINGSTON PARISH', 'MADISON PARISH',
       'MOREHOUSE PARISH', 'NATCHITOCHES PARISH', 'ORLEANS PARISH',
       'OUACHITA PARISH', 'PLAQUEMINES PARISH', 'POINTE COUPEE PARISH',
       'RAPIDES PARISH', 'RED RIVER PARISH', 'RICHLAND PARISH',
       'SABINE PARISH', 'ST. BERNARD PARISH', 'ST. CHARLES PARISH',

In [97]:
fl_ship_pop = fl_ship_fips.merge(fl_pop_fips, how='left', on=['FIPS Code', 'YEAR'])


array(['Arizona', 'Florida', nan, 'South Carolina'], dtype=object)

In [96]:
buyer_na = fl_ship_pop.loc[fl_ship_pop.CTYNAME.isna(), ['BUYER_STATE', 'BUYER_COUNTY']]
buyer_na['C, S'] = buyer_na['BUYER_COUNTY'] +', ' +buyer_na['BUYER_STATE']
buyer_na['C, S'].unique()

array(['DE SOTO, FL', 'SAINT JOHNS, FL', 'SAINT LUCIE, FL', 'ACADIA, LA',
       'ALLEN, LA', 'ASCENSION, LA', 'ASSUMPTION, LA', 'AVOYELLES, LA',
       'BEAUREGARD, LA', 'BIENVILLE, LA', 'BOSSIER, LA', 'CADDO, LA',
       'CALCASIEU, LA', 'CALDWELL, LA', 'CAMERON, LA', 'CATAHOULA, LA',
       'CLAIBORNE, LA', 'CONCORDIA, LA', 'DE SOTO, LA',
       'EAST BATON ROUGE, LA', 'EAST CARROLL, LA', 'EAST FELICIANA, LA',
       'EVANGELINE, LA', 'FRANKLIN, LA', 'GRANT, LA', 'IBERIA, LA',
       'IBERVILLE, LA', 'JACKSON, LA', 'JEFFERSON, LA',
       'JEFFERSON DAVIS, LA', 'LA SALLE, LA', 'LAFAYETTE, LA',
       'LAFOURCHE, LA', 'LINCOLN, LA', 'LIVINGSTON, LA', 'MADISON, LA',
       'MOREHOUSE, LA', 'NATCHITOCHES, LA', 'ORLEANS, LA', 'OUACHITA, LA',
       'PLAQUEMINES, LA', 'POINTE COUPEE, LA', 'RAPIDES, LA',
       'RED RIVER, LA', 'RICHLAND, LA', 'SABINE, LA', 'SAINT BERNARD, LA',
       'SAINT CHARLES, LA', 'SAINT HELENA, LA', 'SAINT JAMES, LA',
       'SAINT LANDRY, LA', 'SAINT MARTIN, LA'